In [1]:
from natasha import AddressExtractor

In [2]:
import pymorphy2
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd

In [3]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('russian')

In [4]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    lower = (word.lower() for word in nltk.wordpunct_tokenize(text))
    pattern = re.compile("[а-я+0-9-]+")
    words = pattern.findall(" ".join(lower))
    norm_form = (morph.parse(word)[0].normal_form for word in words)
    tokens = [word for word in text if word not in stop_words]
    return " ".join(tokens)

In [5]:
morph = pymorphy2.MorphAnalyzer()
stop_words = stopwords.words("russian")

def process_text(text):
    tok = text.split(' ')
    pattern = re.compile("[А-я+0-9-]+")
    words = pattern.findall(" ".join(tok))
    #norm_form = (stemmer.stem(word) for word in words)
    tokens = [word for word in words if word not in stop_words]
    return " ".join(tokens)

In [6]:
data = pd.read_csv('data/post_data.csv')
data = data.dropna()

In [7]:
df = data[:]

In [13]:
text = 'Мойка на ПЛАНЕРНОЙ объявляет об АКЦИЯХ :\n\n✨Покрытие Тефлон (защитное покрытие блеск+гидрофоб)\nСедан 1 000 рублей\nКроссовер 1 250 рублей\nДжип 1 500 рублей\n(В стоимость включена мойка авто, обезжиривание, полимерное покрытие с тефлоном)\nАктуально в дождь, т.к. имеет высокий гидрофобный эффект и машина дольше остаётся чистой!!!\n\n✨ защитное гидрофобное покрытие для стёкол\nЛобовое стекло 1 200 рублей\nЛобовое стекло\n+ передние боковые 1 500 рублей\n(Имеет сильный гидрофобный эффект,\nчто обеспечивает комфортное вождение в условиях дождя! Срок службы 6 месяцев! )\n\n✨Мойка ЛЮКС\nСедан/кроссовер 500 руб\nДжип 700 руб\n(В стоимость входит мойка СТАНДАРТ, пылесос салона, протирание всех пластиковых поверхностей, нанесение полиролей для пластика, мойка стёкол )\n\nНа остальные виды работ уточняйте цены по тел: 8-921-750-35-86 или в\nличные сообщения https://vk.com/milamaier\n\n📍Адрес: Санкт-Петербург, ул. Планерная, 24'

In [20]:
f = open('street_list_spb.txt', 'r')
l = [line.strip() for line in f]

In [23]:
k = 0
o = []
for i in list(df['text']):
    sample = process_text(i)
    p = []
    for j in l:
        if sample.find(stemmer.stem(j)) > 0:
            p.append(j)
    o.append(p)

In [25]:
d = 0
for i in o:
    if len(i)>0:
        d+=1
print(d)

836


In [15]:
extractor = AddressExtractor()
sample = process_text(text)
matches = extractor(process_text(sample))
for match in matches:
    print(match.fact)

Address(parts=[Settlement(name='Санкт-Петербург', type=None), Street(name='Планерная', type='улица'), Building(number='24', type=None)])


In [10]:
from tqdm.notebook import tqdm
import time

In [ ]:
k = 0
f = open('street.txt', 'w')
extractor = AddressExtractor()
for i in tqdm(list(df['text'])):
    sample = process_text(i)
    matches = extractor(sample)
    for match in matches:
        s = ''
        for j in range(0, len(match.fact.parts)):
            try:
                if match.fact.parts[j].type is None:
                    try:
                        s += match.fact.parts[j].number
                        continue
                    except:
                        continue
                try:
                    if match.fact.parts[j].number is None:
                        break
                    s += match.fact.parts[j].type + ' ' + match.fact.parts[j].number + ' '
                except:
                    if match.fact.parts[j].name is None:
                        continue
            except:
                continue
                s += match.fact.parts[j].type + ' ' + match.fact.parts[j].name + ' '
        f.write(s + '\n')
    if len(matches)>0:
        k += 1
f.close()

In [ ]:
i

In [141]:
print(k)

122


In [112]:
process_text(text)

'На Комсомола улице дома 10 уехал светофор второй середине перекрестка'

In [124]:
extractor = AddressExtractor()
matches = extractor(process_text(list(df['text'])[0]))
for match in matches:
    print(match.span, match.fact)

In [125]:
len(matches)

0

In [94]:
match.fact.parts

[Street(name='Комсомола',
        type='улица'), Building(number='10',
          type='дом')]

In [183]:
s = match.fact.parts[0].name + ' ' + match.fact.parts[0].type

In [184]:
for i in range(0, len(match.fact.parts)):
    if match.fact.parts[i].type is None:
        s += match.fact.parts[i].number
    else:
        s += match.fact.parts[i].type + ' ' + match.fact.parts[i].number + ' '

TypeError: can only concatenate str (not "NoneType") to str

In [185]:
s

'Лиговском проспект'

In [82]:
match.fact.parts[2].type is None

True